#Using Auto ML to classify Diabetic patients

In [ ]:
pip show azureml-train-automl

Name: azureml-train-automl
Version: 1.48.0
Summary: Used for automatically finding the best machine learning model and its parameters.
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: azureml-train-automl-runtime, azureml-automl-runtime, azureml-train-automl-client, azureml-dataset-runtime, azureml-automl-core
Required-by: azureml-automl-dnn-nlp
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.48.0 to work with dsba6190ml


In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')


# Split the dataset into training and validation subsets
diabetes_ds = ws.datasets.get("diabetes dataset")
train_ds, test_ds = diabetes_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

Dataset already registered.
Data ready!


In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gamma-amarripe-automl"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
import azureml.train.automl.utilities as automl_utils

for metric in automl_utils.get_primary_metrics('classification'):
    print(metric)

accuracy
AUC_weighted
average_precision_score_weighted
norm_macro_recall
precision_score_weighted


In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='Diabetic',
                             iterations=4,
                             primary_metric = 'accuracy',
                             max_concurrent_iterations=2,
                             featurization='auto'
                             )

print("Ready for Auto ML run.")
     

Ready for Auto ML run.


In [ ]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'mslearn-diabetes-automl-sdk')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting Auto ML experiment...
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
mslearn-diabetes-automl-sdk,AutoML_c8c8ea84-30b2-4eca-86eb-70b1d93365ba,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
mslearn-diabetes-automl-sdk,AutoML_c8c8ea84-30b2-4eca-86eb-70b1d93365ba,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

***********************************************************************

{'runId': 'AutoML_c8c8ea84-30b2-4eca-86eb-70b1d93365ba',
 'target': 'gamma-amarripe-automl',
 'status': 'Completed',
 'startTimeUtc': '2023-03-25T20:12:07.659254Z',
 'endTimeUtc': '2023-03-25T20:49:12.13214Z',
 'services': {},
 'properties': {'num_iterations': '4',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'gamma-amarripe-automl',
  'AMLSettingsJsonString': '{"path":null,"name":"Automated ML Experiment","subscription_id":"0bb59590-d012-407d-a545-7513aae8c4a7","resource_group":"dsba6190-class-rg","workspace_name":"dsba6190ml","region":"eastus","compute_target":"gamma-amarripe-automl","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_parallel_run":false,"num_procs":null,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enabl

In [ ]:
for run in automl_run.get_children():
    print('Run ID', run.id)
    for metric in run.get_metrics():
        print('\t', run.get_metrics(metric))

Run ID AutoML_c8c8ea84-30b2-4eca-86eb-70b1d93365ba_2
	 {'norm_macro_recall': 0.8748128411060732}
	 {'average_precision_score_weighted': 0.9881229516289634}
	 {'recall_score_weighted': 0.9453349773941636}
	 {'recall_score_macro': 0.9374064205530366}
	 {'AUC_macro': 0.9875373750066234}
	 {'precision_score_macro': 0.93982148185134}
	 {'AUC_weighted': 0.9875373750066233}
	 {'balanced_accuracy': 0.9374064205530366}
	 {'average_precision_score_micro': 0.98922331457827}
	 {'average_precision_score_macro': 0.9854426139147703}
	 {'f1_score_weighted': 0.9452597304994242}
	 {'AUC_micro': 0.988956648116079}
	 {'accuracy': 0.9453349773941636}
	 {'f1_score_macro': 0.9385961999319525}
	 {'precision_score_weighted': 0.945213206940457}
	 {'log_loss': 0.13532807040573105}
	 {'weighted_accuracy': 0.9517268377582002}
	 {'matthews_correlation': 0.877224577993405}
	 {'recall_score_micro': 0.9453349773941636}
	 {'f1_score_micro': 0.9453349773941636}
	 {'precision_score_micro': 0.9453349773941636}
	 {'confusi

In [ ]:

best_run, fitted_model = automl_run.get_output()
print(best_run)
print('\nBest Model Definition:')
print(fitted_model)
print('\nBest Run Transformations:')
for step in fitted_model.named_steps:
    print(step)
print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: mslearn-diabetes-automl-sdk,
Id: AutoML_c8c8ea84-30b2-4eca-86eb-70b1d93365ba_2,
Type: azureml.scriptrun,
Status: Completed)

Best Model Definition:
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/moun...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('1', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('xgboostclassifier', XGBoostClassifier(n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('l

In [ ]:
from azureml.interpret import ExplanationClient

client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation(raw=False)
print(engineered_explanations.get_feature_importance_dict())

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


{'Pregnancies_CharGramCountVectorizer_0': 1.183563410216419, 'Age_MeanImputer': 1.0296053630165958, 'BMI_MeanImputer': 0.7504240663891858, 'Pregnancies_CharGramCountVectorizer_1': 0.6054966299842207, 'SerumInsulin_MeanImputer': 0.6039335690649478, 'PlasmaGlucose_MeanImputer': 0.5267300998275708, 'TricepsThickness_MeanImputer': 0.3388523821774129, 'DiastolicBloodPressure_MeanImputer': 0.2706481177820328, 'DiabetesPedigree_MeanImputer': 0.16247908070687786, 'Pregnancies_CharGramCountVectorizer_7': 0.11046522334932957, 'Pregnancies_CharGramCountVectorizer_2': 0.09646530179795205, 'Pregnancies_CharGramCountVectorizer_5': 0.09498058207450435, 'Pregnancies_CharGramCountVectorizer_4': 0.08041629161456178, 'Pregnancies_CharGramCountVectorizer_6': 0.05691390920807304, 'Pregnancies_CharGramCountVectorizer_3': 0.034629180082967004, 'PatientID_MeanImputer': 0.03267007118739496, 'Column1_MeanImputer': 0.0318964594613034, 'Pregnancies_CharGramCountVectorizer_10': 0.01768088297403924, 'Pregnancies_Ch

In [ ]:

print(
    "You can visualize the engineered explanations under the 'Explanations (preview)' tab in the AutoML run at:-\n"
    + automl_run.get_portal_url()
)

You can visualize the engineered explanations under the 'Explanations (preview)' tab in the AutoML run at:-
https://ml.azure.com/runs/AutoML_c8c8ea84-30b2-4eca-86eb-70b1d93365ba?wsid=/subscriptions/0bb59590-d012-407d-a545-7513aae8c4a7/resourcegroups/dsba6190-class-rg/workspaces/dsba6190ml&tid=88d59d7d-aecb-41b2-90c5-55595de02536


In [ ]:

from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='gamma_amarripe_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')
     

gamma_amarripe_model version: 1
	 Training context : Auto ML
	 AUC : 0.9875373750066233
	 Accuracy : 0.9453349773941636


Production_Diabetes_Ensemble version: 1
	 Training Context : Inline Training
	 azureml.datastoreId : /subscriptions/0bb59590-d012-407d-a545-7513aae8c4a7/resourceGroups/dsba6190-class-rg/providers/Microsoft.MachineLearningServices/workspaces/dsba6190ml/datastores/workspaceartifactstore


sgupta46_xgb_diabetes_model_new version: 5
	 Training context : Inline Training


sgupta46_stack_diabetes_model_new version: 4
	 Training context : Inline Training


sgupta46_stack_diabetes_model_new version: 3
	 Training context : Inline Training


sgupta46_stack_diabetes_model_new version: 2
	 Training context : Inline Training


sgupta46_stack_diabetes_model_new version: 1
	 Training context : Inline Training


sgupta46_xgb_diabetes_model_new version: 4
	 Training context : Inline Training


inst_diabetes_model version: 5
	 Training context : Inline Training


sakshi_test123 versi